# Simple analysis of opioid overdose mortality from 1999 - 2016. A widget was created to visualize by state, race, and age range.

You can just read read the csv file into the script. 

This is one of my first data projects that I completed when I started learning in ~2019. Very simple project!


In [ ]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Read csv file
df_opioid = pd.read_csv('Compressed_Mortality_1999-2016.csv')

ALL = 'All'

# Function to sort unique data for visualization
def unique_sorted_values_plus_all(series):
    """
    Helper function to make a list from a series.
    Used to populate dropdowns
    """
    item_list = series.unique().tolist()  # find unique values of series, cast to list (important for dropdown )
    item_list.sort()
    item_list.insert(0, ALL)
    return item_list

# create output
output = widgets.Output()
plot_output = widgets.Output()  # PLOT the data


# create dropdowns
dropdown_state = widgets.Dropdown(options = unique_sorted_values_plus_all(df_opioid.State), description='State')
dropdown_race = widgets.Dropdown(options = unique_sorted_values_plus_all(df_opioid.Race), description='Race')
dropdown_ages = widgets.Dropdown(options = unique_sorted_values_plus_all(df_opioid.Ages), description='Age Range')

def filter_df_opioid(state, race, ages): ## arguments for code to execute
    """
    Filters dataframe by State, Race, Ages
    """
    output.clear_output() ## updates every time user filters data
    plot_output.clear_output()

    filter_df_opioid = df_opioid ## dataframe is equal to the filtered dataframe
    if (state != ALL): ## if State is not ALL... then filter by State
        filter_df_opioid = filter_df_opioid[filter_df_opioid.State == state] ## filter by State
    if (race != ALL):
        filter_df_opioid = filter_df_opioid[filter_df_opioid.Race == race]
    if (ages != ALL):
        filter_df_opioid = filter_df_opioid[filter_df_opioid.Ages == ages]
    with output: # Outputs Dataset
        display(filter_df_opioid)
    if filter_df_opioid.size != 0: 
        with plot_output:  # Plot Dataset
            g = sns.catplot(kind = 'point', x = 'Year', 
                                y = 'Crude Death Rate (Per 100k Persons)', 
                                hue = 'State', col = 'Gender', data = filter_df_opioid)
            g.fig.set_figwidth(19)
            g.fig.set_figheight(11)
            plt.show()
    else:
        print("No data given. Please try a different filter.")

# event handler
def dropdown_state_eventhandler(change):
    filter_df_opioid(change.new, dropdown_race.value, dropdown_ages.value)

def dropdown_race_eventhandler(change):
    filter_df_opioid(dropdown_state.value, change.new, dropdown_ages.value)

def dropdown_ages_eventhandler(change):
    filter_df_opioid(dropdown_state.value, dropdown_race.value, change.new)

# bind to events
dropdown_state.observe(dropdown_state_eventhandler, names='value')
dropdown_race.observe(dropdown_race_eventhandler, names='value')
dropdown_ages.observe(dropdown_ages_eventhandler, names='value')

# displays dropdowns & organizes orientation of dashboard
display_input_widgets = widgets.HBox([dropdown_state, dropdown_race, dropdown_ages])
tab = widgets.Tab([output, plot_output])
tab.set_title(0, 'Dataset')
tab.set_title(1, 'Plotted Data')
dashboard = widgets.VBox([display_input_widgets, tab])
display(dashboard)